In [ ]:
!pip install wandb


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.3/184.3 KB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 189.1/189.1 KB 12.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 4.5 MB/s eta 0:00:00
  Created wheel for pathtools: filename=pathtools-0.1.2-py3-none-any.whl size=8807 sha256=b47b8bd11b4a40734286bd717cb4bae505879d0fc3001f6feb5723188ee3f7cb
  Stored in directory: /root/.cache/pip/wheels/b7/0a/67/ada2a22079218c75a88361c0782855cc72aebc4d18d0289d05
Successfully built pathtools


Import libraries

In [ ]:
import numpy as np
import numpy as np
import scipy as sp
import wandb
import time
import wandb
import numpy as np
import matplotlib.pyplot as plt
from keras.datasets import fashion_mnist




Activation functions and their derivatives


In [ ]:

def sigmoid(z):
    return 1.0 / (1 + np.exp(-(z)))

def tanh(z):
    return np.tanh(z)

def sin(z):
    return np.sin(z)

def relu(z):
    return (z>0)*(z) + ((z<0)*(z)*0.01)
  
def softmax(Z):
    return np.exp(Z) / np.sum(np.exp(Z))

def sigmoid_derivative(z):
    return  (1.0 / (1 + np.exp(-(z))))*(1 -  1.0 / (1 + np.exp(-(z))))

def tanh_derivative(z):
    return 1 - np.tanh(z) ** 2

def relu_derivative(z):
    return (z>0)*np.ones(z.shape) + (z<0)*(0.01*np.ones(z.shape) )

In [ ]:


class Feed_Forward_NN __init__(
        self, 
        hidden_layers, 
        hidden_neurons, 
        Xtrain_ini, 
        ytrain_ini  
        train_n, 
        Xval_ini 
        yval_ini 
        val_n,
        Xtest_ini 
        ytest_ini 
        N_test,        
        optimizer,
        batch_size,
        weight_decay,
        learning_rate,
        maxNo_epochh,
        activation,
        initializer,
        loss

        '''
         Initializing the Feed_Forward_NN class.
        '''
        
        self.classes = np.max(ytrain_ini) + 1  # number of classes
        self.hidden_layers = hidden_layers
        self.hidden_neurons = hidden_neurons
        self.op = self.classes  #
        self.img_height = Xtrain_ini.shape[1]
        self.img_width = Xtrain_ini.shape[2]
        self.size_imgflat = self.img_height * self.img_width
        self.layers = ([self.size_imgflat]+ hidden_layers * [hidden_neurons]+ [self.op])
        self.train_n = train_n
        self.val_n = val_n
        self.N_test = N_test
        


        self.X_train = np.transpose(Xtrain_ini.reshape(Xtrain_ini.shape[0], Xtrain_ini.shape[1] * Xtrain_ini.shape[2]))  
        self.X_test = np.transpose(Xtest_ini.reshape(Xtest_ini.shape[0], Xtest_ini.shape[1] * Xtest_ini.shape[2]))
        self.X_val = np.transpose(Xval_ini.reshape(Xval_ini.shape[0], Xval_ini.shape[1] * Xval_ini.shape[2]))
    

        self.X_train = self.X_train / 255
        self.X_test = self.X_test / 255
        self.X_val = self.X_val / 255
        
        self.y_train = self.one_hot_encode(ytrain_ini)
        self.y_val = self.one_hot_encode(yval_ini)
        self.y_test = self.one_hot_encode(ytest_ini)
       
#Activation Functions

        self.activation_fns = {"SIGMOID": sigmoid, "TANH": tanh, "RELU": relu}
        self.dv_activation_fns = {
            "SIGMOID": sigmoid_derivative,
            "TANH": tanh_derivative,
            "RELU": relu_derivative,
        }

#Initializers

        self.Initializers = {
            "XAVIER": self.Xavier,
            "RANDOM": self.Random,
            "HE": self.He
        }

#Optimizers

        self.Optimizers = {
            "SGD": self.sgd_mb,
            "MGD": self.mgd,
            "Ng: self.Ng,
            "RMSPROP": self.rmsProp,
            "ADAM": self.adam,
            "NADAM": self.nadam,
        }
        
        self.activation = self.activation_fns[activation] 
        self.der_activation = self.dv_activation_fns[activation]
        self.optimizer = self.Optimizers[optimizer]
        self.initializer = self.Initializers[initializer]
        self.loss_function = loss
        self.maxNo_epoch = maxNo_epoch
        self.batch_size = batch_size
        self.learning_rate = learning_rate
        
        self.weights, self.biases = self.initialize_NN(self.layers)


    def one_hot_encode(self, ytrain_ini):
        y_data= np.zeros((self.classes, ytrain_ini.shape[0]))
        for i in range(ytrain_ini.shape[0]):
            value = ytrain_ini[i]
            y_data[int(value)][i] = 1.0
        return y_data

    # Loss functions
    def MSE_Loss(self, Y_true, y_predict):
        MSE = np.mean((Y_true - y_predict) ** 2)
        return MSE

    def CE_loss(self, Y_true, y_predict):
        CE = [-Y_true[i] * np.log(y_predict[i]) for i in range(len(y_predict))]
        crossEntropy = np.mean(CE)
        return crossEntropy

    def L2R_Loss(self, weight_decay):
        ALPHA = weight_decay
        return ALPHA * np.sum([
                np.linalg.norm(self.weights[str(i + 1)]) ** 2
                for i in range(len(self.weights))
            ]
        )


    def accuracy(self, Y_true, y_predict, data_size):
        y_actual = []
        y_predicted = []
        ctr= 0
        for i in range(data_size):
            y_actual.append(np.argmax(Y_true[:, i]))
            y_predicted.append(np.argmax(y_predict[:, i]))
            if y_actual[i] == y_predicted[i]:
                ctr+= 1
        accuracy = ctr/ data_size
        return accuracy, y_actual, y_predicted

    def Xavier(self, size):
        ip_dimension = size[1]
        op_dimension = size[0]
        std_dev_xavier = np.sqrt(2 / (ip_dimension + op_dimension))
        return np.random.normal(0, std_dev_xavier, size=(op_dimension, ip_dimension))

    def Random(self, size):
        ip_dimension = size[1]
        op_dimension = size[0]
        return np.random.normal(0, 1, size=(op_dimension, ip_dimension))


    def He(self,size):
        ip_dimension = size[1]
        op_dimension = size[0]
        std_dev_he = np.sqrt(2 / (ip_dimension))
        return np.random.normal(0, 1, size=(op_dimension, ip_dimension)) * std_dev_he


    def initialize_NN(self, layers):
        weights = {}
        biases = {}
        num_layers = len(layers)
        for l in range(0, num_layers - 1):
            W = self.initializer(size=[layers[l + 1], layers[l]])
            b = np.zeros((layers[l + 1], 1))
            weights[str(l + 1)] = W
            biases[str(l + 1)] = b
        return weights, biases



SyntaxError: ignored

In [ ]:

    def FF_propagate(self, X_train_set, weights, biases):
        """
        Returns the neural network given input data, weights, biases.
        Arguments:
                 : X - input matrix
                 : Weights  - Weights matrix
                 : biases - Bias vectors 
        """
        # Number of layers = length of weight matrix + 1
        num_layers = len(weights) + 1
        # A - Preactivations
        # H - Activations
        X = X_train_set
        H = {}
        A = {}
        H["0"] = X
        A["0"] = X
        for l in range(0, num_layers - 2):
            if l == 0:
                W = weights[str(l + 1)]
                b = biases[str(l + 1)]
                A[str(l + 1)] = np.add(np.matmul(W, X), b)
                H[str(l + 1)] = self.activation(A[str(l + 1)])
            else:
                W = weights[str(l + 1)]
                b = biases[str(l + 1)]
                A[str(l + 1)] = np.add(np.matmul(W, H[str(l)]), b)
                H[str(l + 1)] = self.activation(A[str(l + 1)])

        # Here the last layer is not activated as it is a regression problem
        W = weights[str(num_layers - 1)]
        b = biases[str(num_layers - 1)]
        A[str(num_layers - 1)] = np.add(np.matmul(W, H[str(num_layers - 2)]), b)
        # Y = softmax(A[-1])
        Y = softmax(A[str(num_layers - 1)])
        H[str(num_layers - 1)] = Y
        return Y, H, A

    def back_propagate(
        self, Y, H, A, y_train_set, weight_decay=0
    ):

        ALPHA = weight_decay
        gr_weights = []
        gr_biases = []
        num_layers = len(self.layers)

        if self.loss_function == "CROSS":
            globals()["grad_a" + str(num_layers - 1)] = -(y_train_set - Y)
        elif self.loss_function == "MSE":
            globals()["grad_a" + str(num_layers - 1)] = np.multiply(
                2 * (Y - y_train_set), np.multiply(Y, (1 - Y))
            )

        for l in range(num_layers - 2, -1, -1):

            if ALPHA != 0:
                globals()["grad_W" + str(l + 1)] = (
                    np.outer(globals()["grad_a" + str(l + 1)], H[str(l)])
                    + ALPHA * self.weights[str(l + 1)]
                )
            elif ALPHA == 0:
                globals()["grad_W" + str(l + 1)] = np.outer(
                    globals()["grad_a" + str(l + 1)], H[str(l)]
                )
            globals()["grad_b" + str(l + 1)] = globals()["grad_a" + str(l + 1)]
            gr_weights.append(globals()["grad_W" + str(l + 1)])
            gr_biases.append(globals()["grad_b" + str(l + 1)])
            if l != 0:
                globals()["grad_h" + str(l)] = np.matmul(
                    self.weights[str(l + 1)].transpose(),
                    globals()["grad_a" + str(l + 1)],
                )
                globals()["grad_a" + str(l)] = np.multiply(
                    globals()["grad_h" + str(l)], self.der_activation(A[str(l)])
                )
            elif l == 0:

                globals()["grad_h" + str(l)] = np.matmul(
                    self.weights[str(l + 1)].transpose(),
                    globals()["grad_a" + str(l + 1)],
                )
                globals()["grad_a" + str(l)] = np.multiply(
                    globals()["grad_h" + str(l)], (A[str(l)])
                )
        return gr_weights, gr_biases


    def predict(self,X,len_dataset):
        y_predict = []        
        for i in range(len_dataset):

            Y, H, A = self.FF_propagate(
                X[:, i].reshape(self.size_imgflat, 1),
                self.weights,
                self.biases,
            )

            y_predict.append(Y.reshape(self.classes,))
        y_predict = np.array(y_predict).transpose()
        return y_predict

    def sgd(self, epochs, len_dataset, learning_rate, weight_decay=0):
        
        training_loss = []
        trainingaccuracy= []
        validationaccuracy = []
        
        num_layers = len(self.layers)

        X_train = self.X_train[:, :len_dataset]
        y_train = self.y_train[:, :len_dataset]

        for epoch in range(epochs):
            start_time = time.time()
            idx = np.random.shuffle(np.arange(len_dataset))
            X_train = X_train[:, idx].reshape(self.size_imgflat, len_dataset)
            y_train = y_train[:, idx].reshape(self.classes, len_dataset)

            CE = []
            dW= [
                np.zeros((self.layers[l + 1], self.layers[l]))
                for l in range(0, len(self.layers) - 1)
            ]
            db = [
                np.zeros((self.layers[l + 1], 1))
                for l in range(0, len(self.layers) - 1)
            ]

            for i in range(len_dataset):

                Y, H, A = self.FF_propagate(
                    X_train[:, i].reshape(self.size_imgflat, 1),
                    self.weights,
                    self.biases,
                )
                grad_weights, grad_biases = self.back_propagate(
                    Y, H, A, y_train[:, i].reshape(self.classes, 1)
                )
                dW= [
                    grad_weights[num_layers - 2 - i] for i in range(num_layers - 1)
                ]
                db = [
                    grad_biases[num_layers - 2 - i] for i in range(num_layers - 1)
                ]


                CE.append(
                    self.CE_loss(
                        self.y_train[:, i].reshape(self.classes, 1), Y
                    )
                    + self.L2R_Loss(weight_decay)
                )

                self.weights = {
                    str(i + 1): (self.weights[str(i + 1)] - learning_rate * dw[i])
                    for i in range(len(self.weights))
                }
                self.biases = {
                    str(i + 1): (self.biases[str(i + 1)] - learning_rate * db[i])
                    for i in range(len(self.biases))
                }

            elapsed = time.time() - start_time
            y_predict = self.predict(self.X_train, self.train_n)
            training_loss.append(np.mean(CE))
            trainingaccuracy.append(self.accuracy(y_train, y_predict, len_dataset)[0])
            validationaccuracy.append(self.accuracy(self.y_val, self.predict(self.X_val, self.val_n), self.val_n)[0])
            
            print(
                        "Epoch: %d, Loss: %.3e, Training accuracy:%.2f, Validation Accuracy: %.2f, Time: %.2f, Learning Rate: %.3e"
                        % (
                            epoch,
                            training_loss[epoch],
                            trainingaccuracy[epoch],
                            validationaccuracy[epoch],
                            elapsed,
                            self.learning_rate,
                        )
                    )

            wandb.log({'loss':np.mean(CE), 'trainingaccuracy':trainingaccuracy[epoch], 'validationaccuracy':validationaccuracy[epoch],'epoch':epoch, })
        
        return training_loss, trainingaccuracy, validationaccuracy, y_predict


      
    def sgd_mb(self, epochs,len_dataset, batch_size, learning_rate, weight_decay = 0):

        X_train = self.X_train[:, :len_dataset]
        y_train = self.y_train[:, :len_dataset]        

        training_loss = []
        trainingaccuracy = []
        validationaccuracy = []
        
        num_layers = len(self.layers)
        npoints = 0


        for epoch in range(epochs):
            start_time = time.time()
            idx = np.random.shuffle(np.arange(len_dataset))
            X_train = X_train[:, idx].reshape(self.size_imgflat, len_dataset)
            y_train = y_train[:, idx].reshape(self.classes, len_dataset)
            
            CE = []
            
            dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
            db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]

            for i in range(len_dataset):
                
                Y,H,A = self.FF_propagate(X_train[:,i].reshape(self.size_imgflat,1), self.weights, self.biases) 
                grad_weights, grad_biases = self.back_propagate(Y,H,A,y_train[:,i].reshape(self.classes,1))
                
                dW= [grad_weights[num_layers-2 - i] + dw[i] for i in range(num_layers - 1)]
                db = [grad_biases[num_layers-2 - i] + db[i] for i in range(num_layers - 1)]
                
                #y_predict.append(Y.reshape(self.classes,))
                CE.append(self.CE_loss(self.y_train[:,i].reshape(self.classes,1), Y) + self.L2R_Loss(weight_decay))
                
                npoints +=1
                
                if int(npoints) % batch_size == 0:
                    
                    
                    self.weights = {str(i+1):(self.weights[str(i+1)] - learning_rate*dw[i]/batch_size) for i in range(len(self.weights))} 
                    self.biases = {str(i+1):(self.biases[str(i+1)] - learning_rate*db[i]) for i in range(len(self.biases))}
                    
                    dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
                    db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
            
            elapsed = time.time() - start_time
            y_predict = self.predict(self.X_train, self.train_n)
            training_loss.append(np.mean(CE))
            trainingaccuracy.append(self.accuracy(y_train, y_predict, len_dataset)[0])
            validationaccuracy.append(self.accuracy(self.y_val, self.predict(self.X_val, self.val_n), self.val_n)[0])

            print(
                        "Epoch: %d, Loss: %.3e, Training accuracy:%.2f, Validation Accuracy: %.2f, Time: %.2f, Learning Rate: %.3e"
                        % (
                            epoch,
                            training_loss[epoch],
                            trainingaccuracy[epoch],
                            validationaccuracy[epoch],
                            elapsed,
                            self.learning_rate,
                        )
                    )
                    
            wandb.log({'loss':np.mean(CE), 'trainingaccuracy':trainingaccuracy[epoch], 'validationaccuracy':validationaccuracy[epoch],'epoch':epoch })
            
        return training_loss, trainingaccuracy, validationaccuracy, y_predict



    def mgd(self, epochs,len_dataset, batch_size, learning_rate, weight_decay = 0):
        GAMMA = 0.9

        X_train = self.X_train[:, :len_dataset]
        y_train = self.y_train[:, :len_dataset]        

        
        training_loss = []
        trainingaccuracy = []
        validationaccuracy = []
        
        num_layers = len(self.layers)
        prev_vw = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
        prev_vb = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
        npoints = 0
        for epoch in range(epochs):
            start_time = time.time()
            idx = np.random.shuffle(np.arange(len_dataset))
            X_train = X_train[:, idx].reshape(self.size_imgflat, len_dataset)
            y_train = y_train[:, idx].reshape(self.classes, len_dataset)

            CE = []
            #y_predict = []
            dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
            db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
            

            for i in range(len_dataset):
                Y,H,A = self.FF_propagate(self.X_train[:,i].reshape(self.size_imgflat,1), self.weights, self.biases) 
                grad_weights, grad_biases = self.back_propagate(Y,H,A,self.y_train[:,i].reshape(self.classes,1))
                
                dW= [grad_weights[num_layers-2 - i] + dw[i] for i in range(num_layers - 1)]
                db = [grad_biases[num_layers-2 - i] + db[i] for i in range(num_layers - 1)]

                CE.append(self.CE_loss(self.y_train[:,i].reshape(self.classes,1), Y) + self.L2R_Loss(weight_decay))
                
                npoints +=1
                
                if int(npoints) % batch_size == 0:

                    vw = [GAMMA*prev_vw[i] + learning_rate*dw[i]/batch_size for i in range(num_layers - 1)]
                    vb = [GAMMA*prev_vb[i] + learning_rate*db[i]/batch_size for i in range(num_layers - 1)]
                    
                    self.weights = {str(i+1) : (self.weights[str(i+1)] - vw[i]) for i in range(len(self.weights))}
                    self.biases = {str(i+1): (self.biases[str(i+1)] - vb[i]) for i in range(len(self.biases))}

                    prev_vw = vw
                    prev_vb = vb

                    #resetting gradient updates
                    dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
                    db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]

            elapsed = time.time() - start_time
            y_predict = self.predict(self.X_train, self.train_n)
            training_loss.append(np.mean(CE))
            trainingaccuracy.append(self.accuracy(y_train, y_predict, len_dataset)[0])
            validationaccuracy.append(self.accuracy(self.y_val, self.predict(self.X_val, self.val_n), self.val_n)[0])

            print(
                        "Epoch: %d, Loss: %.3e, Training accuracy:%.2f, Validation Accuracy: %.2f, Time: %.2f, Learning Rate: %.3e"
                        % (
                            epoch,
                            training_loss[epoch],
                            trainingaccuracy[epoch],
                            validationaccuracy[epoch],
                            elapsed,
                            self.learning_rate,
                        )
                    )

            wandb.log({'loss':np.mean(CE), 'trainingaccuracy':trainingaccuracy[epoch], 'validationaccuracy':validationaccuracy[epoch],'epoch':epoch })


        return training_loss, trainingaccuracy, validationaccuracy, y_predict


 
 
    def stochastic_ng(self,epochs,len_dataset, learning_rate, weight_decay = 0):
        GAMMA = 0.9

        X_train = self.X_train[:, :len_dataset]
        y_train = self.y_train[:, :len_dataset]        

        training_loss = []
        trainingaccuracy = []
        validationaccuracy = []
        
        num_layers = len(self.layers)
        
        prev_vw = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
        prev_vb = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
        
        for epoch in range(epochs):
            start_time = time.time()
            idx = np.random.shuffle(np.arange(len_dataset))
            X_train = X_train[:, idx].reshape(self.size_imgflat, len_dataset)
            y_train = y_train[:, idx].reshape(self.classes, len_dataset)

            CE = []
            #y_predict = []  
            
            dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
            db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
            
            vw = [GAMMA*prev_vw[i] for i in range(0, len(self.layers)-1)]  
            vb = [GAMMA*prev_vb[i] for i in range(0, len(self.layers)-1)]
                        
            for i in range(len_dataset):
                w_inter = {str(i+1) : self.weights[str(i+1)] - vw[i] for i in range(0, len(self.layers)-1)}
                b_inter = {str(i+1) : self.biases[str(i+1)] - vb[i] for i in range(0, len(self.layers)-1)}
                
                Y,H,A = self.FF_propagate(self.X_train[:,i].reshape(self.size_imgflat,1), w_inter, b_inter) 
                grad_weights, grad_biases = self.back_propagate(Y,H,A,self.y_train[:,i].reshape(self.classes,1))
                
                dW= [grad_weights[num_layers-2 - i] for i in range(num_layers - 1)]
                db = [grad_biases[num_layers-2 - i] for i in range(num_layers - 1)]

                CE.append(self.CE_loss(self.y_train[:,i].reshape(self.classes,1), Y) + self.L2R_Loss(weight_decay))
                            
                vw = [GAMMA*prev_vw[i] + learning_rate*dw[i] for i in range(num_layers - 1)]
                vb = [GAMMA*prev_vb[i] + learning_rate*db[i] for i in range(num_layers - 1)]
        
                self.weights = {str(i+1):self.weights[str(i+1)] - vw[i] for i in range(len(self.weights))} 
                self.biases = {str(i+1):self.biases[str(i+1)] - vb[i] for i in range(len(self.biases))}
                
                prev_vw = vw
                prev_vb = vb
    
            
            elapsed = time.time() - start_time
            #y_predict = np.array(y_predict).transpose()
            y_predict = self.predict(self.X_train, self.train_n)
            training_loss.append(np.mean(CE))
            trainingaccuracy.append(self.accuracy(y_train, y_predict, len_dataset)[0])
            validationaccuracy.append(self.accuracy(self.y_val, self.predict(self.X_val, self.val_n), self.val_n)[0])

            print(
                        "Epoch: %d, Loss: %.3e, Training accuracy:%.2f, Validation Accuracy: %.2f, Time: %.2f, Learning Rate: %.3e"
                        % (
                            epoch,
                            training_loss[epoch],
                            trainingaccuracy[epoch],
                            validationaccuracy[epoch],
                            elapsed,
                            self.learning_rate,
                        )
                    )
                    
            wandb.log({'loss':np.mean(CE), 'trainingaccuracy':trainingaccuracy[epoch], 'validationaccuracy':validationaccuracy[epoch],'epoch':epoch })
        
        return training_loss, trainingaccuracy, validationaccuracy, y_predict
    

    def Ng(self,epochs,len_dataset, batch_size,learning_rate, weight_decay = 0):
        GAMMA = 0.9

        X_train = self.X_train[:, :len_dataset]
        y_train = self.y_train[:, :len_dataset]        


        training_loss = []
        trainingaccuracy = []
        validationaccuracy = []
        
        num_layers = len(self.layers)
        
        prev_vw = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
        prev_vb = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
        
        npoints = 0
        for epoch in range(epochs):
            start_time = time.time()
            idx = np.random.shuffle(np.arange(len_dataset))
            X_train = X_train[:, idx].reshape(self.size_imgflat, len_dataset)
            y_train = y_train[:, idx].reshape(self.classes, len_dataset)

            CE = []
            
            dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
            db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
            
            vw = [GAMMA*prev_vw[i] for i in range(0, len(self.layers)-1)]  
            vb = [GAMMA*prev_vb[i] for i in range(0, len(self.layers)-1)]

            for i in range(len_dataset):
                w_inter = {str(i+1) : self.weights[str(i+1)] - vw[i] for i in range(0, len(self.layers)-1)}
                b_inter = {str(i+1) : self.biases[str(i+1)] - vb[i] for i in range(0, len(self.layers)-1)}
                
                Y,H,A = self.FF_propagate(self.X_train[:,i].reshape(self.size_imgflat,1), w_inter, b_inter) 
                grad_weights, grad_biases = self.back_propagate(Y,H,A,self.y_train[:,i].reshape(self.classes,1))
                
                dW= [grad_weights[num_layers-2 - i] + dw[i] for i in range(num_layers - 1)]
                db = [grad_biases[num_layers-2 - i] + db[i] for i in range(num_layers - 1)]

                #y_predict.append(Y.reshape(self.classes,))
                CE.append(self.CE_loss(self.y_train[:,i].reshape(self.classes,1), Y) + self.L2R_Loss(weight_decay))

                npoints +=1
                
                if int(npoints) % batch_size == 0:                            

                    vw = [GAMMA*prev_vw[i] + learning_rate*dw[i]/batch_size for i in range(num_layers - 1)]
                    vb = [GAMMA*prev_vb[i] + learning_rate*db[i]/batch_size for i in range(num_layers - 1)]
        
                    self.weights ={str(i+1):self.weights[str(i+1)]  - vw[i] for i in range(len(self.weights))}
                    self.biases = {str(i+1):self.biases[str(i+1)]  - vb[i] for i in range(len(self.biases))}
                
                    prev_vw = vw
                    prev_vb = vb

                    dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
                    db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]

    
            
            elapsed = time.time() - start_time

            y_predict = self.predict(self.X_train, self.train_n)
            training_loss.append(np.mean(CE))
            trainingaccuracy.append(self.accuracy(y_train, y_predict, len_dataset)[0])
            validationaccuracy.append(self.accuracy(self.y_val, self.predict(self.X_val, self.val_n), self.val_n)[0])

            print(
                        "Epoch: %d, Loss: %.3e, Training accuracy:%.2f, Validation Accuracy: %.2f, Time: %.2f, Learning Rate: %.3e"
                        % (
                            epoch,
                            training_loss[epoch],
                            trainingaccuracy[epoch],
                            validationaccuracy[epoch],
                            elapsed,
                            self.learning_rate,
                        )
                    )

            wandb.log({'loss':np.mean(CE), 'trainingaccuracy':trainingaccuracy[epoch], 'validationaccuracy':validationaccuracy[epoch],'epoch':epoch })
        
        return training_loss, trainingaccuracy, validationaccuracy, y_predict
    

    
    def rmsProp(self, epochs,len_dataset, batch_size, learning_rate, weight_decay = 0):


        X_train = self.X_train[:, :len_dataset]
        y_train = self.y_train[:, :len_dataset]        

        
        training_loss = []
        trainingaccuracy = []
        validationaccuracy = []
        
        num_layers = len(self.layers)
        EPS, BETA = 1e-8, 0.9
        
        vw = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
        vb = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
        
        npoints = 0        
        for epoch in range(epochs):
            start_time = time.time()
            idx = np.random.shuffle(np.arange(len_dataset))
            X_train = X_train[:, idx].reshape(self.size_imgflat, len_dataset)
            y_train = y_train[:, idx].reshape(self.classes, len_dataset)


            CE = []
            #y_predict = []
                        
            dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
            db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]

            for i in range(len_dataset):
            
                Y,H,A = self.FF_propagate(self.X_train[:,i].reshape(self.size_imgflat,1), self.weights, self.biases) 
                grad_weights, grad_biases = self.back_propagate(Y,H,A,self.y_train[:,i].reshape(self.classes,1))
            
                dW= [grad_weights[num_layers-2 - i] + dw[i] for i in range(num_layers - 1)]
                db = [grad_biases[num_layers-2 - i] + db[i] for i in range(num_layers - 1)]
                
                 CE.append(self.CE_loss(self.y_train[:,i].reshape(self.classes,1), Y) + self.L2R_Loss(weight_decay))            
                npoints +=1
                
                if int(npoints) % batch_size == 0:
                
                    vw = [BETA*vw[i] + (1-BETA)*(dw[i])**2 for i in range(num_layers - 1)]
                    vb = [BETA*vb[i] + (1-BETA)*(db[i])**2 for i in range(num_layers - 1)]

                    self.weights = {str(i+1):self.weights[str(i+1)]  - dw[i]*(learning_rate/np.sqrt(vw[i]+EPS)) for i in range(len(self.weights))} 
                    self.biases = {str(i+1):self.biases[str(i+1)]  - db[i]*(learning_rate/np.sqrt(vb[i]+EPS)) for i in range(len(self.biases))}

                    dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
                    db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
    
            
            elapsed = time.time() - start_time
            #y_predict = np.array(y_predict).transpose()
            y_predict = self.predict(self.X_train, self.train_n)
            training_loss.append(np.mean(CE))
            trainingaccuracy.append(self.accuracy(y_train, y_predict, len_dataset)[0])
            validationaccuracy.append(self.accuracy(self.y_val, self.predict(self.X_val, self.val_n), self.val_n)[0])

            print(
                        "Epoch: %d, Loss: %.3e, Training accuracy:%.2f, Validation Accuracy: %.2f, Time: %.2f, Learning Rate: %.3e"
                        % (
                            epoch,
                            training_loss[epoch],
                            trainingaccuracy[epoch],
                            validationaccuracy[epoch],
                            elapsed,
                            self.learning_rate,
                        )
                    )
                    
            wandb.log({'loss':np.mean(CE), 'trainingaccuracy':trainingaccuracy[epoch], 'validationaccuracy':validationaccuracy[epoch],'epoch':epoch })
        
        return training_loss, trainingaccuracy, validationaccuracy, y_predict  



    def adam(self, epochs,len_dataset, batch_size, learning_rate, weight_decay = 0):
        
        X_train = self.X_train[:, :len_dataset]
        y_train = self.y_train[:, :len_dataset]        

        training_loss = []
        trainingaccuracy = []
        validationaccuracy = []
        num_layers = len(self.layers)
        EPS, BETA1, BETA2 = 1e-8, 0.9, 0.99
        
       mw = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
       mb = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
        
        vw = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
        vb = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]        
        
       mw_hat = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
       mb_hat = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
        
        vw_hat = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
        vb_hat = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]   
        
        npoints = 0 
        for epoch in range(epochs):
            start_time = time.time()
            idx = np.random.shuffle(np.arange(len_dataset))
            X_train = X_train[:, idx].reshape(self.size_imgflat, len_dataset)
            y_train = y_train[:, idx].reshape(self.classes, len_dataset)


            CE = []
            
            dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
            db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
            
           
            for i in range(len_dataset):
                Y,H,A = self.FF_propagate(self.X_train[:,i].reshape(self.size_imgflat,1), self.weights, self.biases) 
                grad_weights, grad_biases = self.back_propagate(Y,H,A,self.y_train[:,i].reshape(self.classes,1))
                
                dW= [grad_weights[num_layers-2 - i] + dw[i] for i in range(num_layers - 1)]
                db = [grad_biases[num_layers-2 - i] + db[i] for i in range(num_layers - 1)]

                CE.append(self.CE_loss(self.y_train[:,i].reshape(self.classes,1), Y) + self.L2R_Loss(weight_decay))                 

                npoints += 1
                ctr= 0
                if int(npoints) % batch_size == 0:
                    ctr+= 1
                
                   mw = [BETA1*m_w[i] + (1-BETA1)*dw[i] for i in range(num_layers - 1)]
                   mb = [BETA1*m_b[i] + (1-BETA1)*db[i] for i in range(num_layers - 1)]
                
                    vw = [BETA2*vw[i] + (1-BETA2)*(dw[i])**2 for i in range(num_layers - 1)]
                    vb = [BETA2*vb[i] + (1-BETA2)*(db[i])**2 for i in range(num_layers - 1)]
                    
                   mw_hat = [m_w[i]/(1-BETA1**(epoch+1)) for i in range(num_layers - 1)]
                   mb_hat = [m_b[i]/(1-BETA1**(epoch+1)) for i in range(num_layers - 1)]            
                
                    vw_hat = [vw[i]/(1-BETA2**(epoch+1)) for i in range(num_layers - 1)]
                    vb_hat = [vb[i]/(1-BETA2**(epoch+1)) for i in range(num_layers - 1)]
                
                    self.weights = {str(i+1):self.weights[str(i+1)] - (learning_rate/np.sqrt(vw[i]+EPS))*m_w_hat[i] for i in range(len(self.weights))} 
                    self.biases = {str(i+1):self.biases[str(i+1)] - (learning_rate/np.sqrt(vb[i]+EPS))*m_b_hat[i] for i in range(len(self.biases))}

                    dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
                    db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]


            elapsed = time.time() - start_time
            #y_predict = np.array(y_predict).transpose()
            y_predict = self.predict(self.X_train, self.train_n)
            training_loss.append(np.mean(CE))
            trainingaccuracy.append(self.accuracy(y_train, y_predict, len_dataset)[0])
            validationaccuracy.append(self.accuracy(self.y_val, self.predict(self.X_val, self.val_n), self.val_n)[0])

            print(
                        "Epoch: %d, Loss: %.3e, Training accuracy:%.2f, Validation Accuracy: %.2f, Time: %.2f, Learning Rate: %.3e"
                        % (
                            epoch,
                            training_loss[epoch],
                            trainingaccuracy[epoch],
                            validationaccuracy[epoch],
                            elapsed,
                            self.learning_rate,
                        )
                    )
                    
            wandb.log({'loss':np.mean(CE), 'trainingaccuracy':trainingaccuracy[epoch], 'validationaccuracy':validationaccuracy[epoch],'epoch':epoch })
        
        return training_loss, trainingaccuracy, validationaccuracy, y_predict


    
    def nadam(self, epochs,len_dataset, batch_size, learning_rate, weight_decay = 0):

        X_train = self.X_train[:, :len_dataset]
        y_train = self.y_train[:, :len_dataset]        

        
        training_loss = []
        trainingaccuracy = []
        validationaccuracy = []
        num_layers = len(self.layers)
        
        GAMMA, EPS, BETA1, BETA2 = 0.9, 1e-8, 0.9, 0.99

       mw = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
       mb = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
        
        vw = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
        vb = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]        

       mw_hat = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
       mb_hat = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
        
        vw_hat = [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
        vb_hat = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)] 

        npoints = 0 
        
        
        for epoch in range(epochs):
            start_time = time.time()
            idx = np.random.shuffle(np.arange(len_dataset))
            X_train = X_train[:, idx].reshape(self.size_imgflat, len_dataset)
            y_train = y_train[:, idx].reshape(self.classes, len_dataset)

            CE = []
            #y_predict = []

            dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
            db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]

            for i in range(len_dataset):

                Y,H,A = self.FF_propagate(self.X_train[:,i].reshape(self.size_imgflat,1), self.weights, self.biases) 
                grad_weights, grad_biases = self.back_propagate(Y,H,A,self.y_train[:,i].reshape(self.classes,1))

                dW= [grad_weights[num_layers-2 - i] + dw[i] for i in range(num_layers - 1)]
                db = [grad_biases[num_layers-2 - i] + db[i] for i in range(num_layers - 1)]

                #y_predict.append(Y.reshape(self.classes,))
                CE.append(self.CE_loss(self.y_train[:,i].reshape(self.classes,1), Y) + self.L2R_Loss(weight_decay))   
                npoints += 1
                
                if npoints % batch_size == 0:
                    
                   mw = [BETA1*m_w[i] + (1-BETA1)*dw[i] for i in range(num_layers - 1)]
                   mb = [BETA1*m_b[i] + (1-BETA1)*db[i] for i in range(num_layers - 1)]
                    
                    vw = [BETA2*vw[i] + (1-BETA2)*(dw[i])**2 for i in range(num_layers - 1)]
                    vb = [BETA2*vb[i] + (1-BETA2)*(db[i])**2 for i in range(num_layers - 1)]
                    
                   mw_hat = [m_w[i]/(1-BETA1**(epoch+1)) for i in range(num_layers - 1)]
                   mb_hat = [m_b[i]/(1-BETA1**(epoch+1)) for i in range(num_layers - 1)]            
                    
                    vw_hat = [vw[i]/(1-BETA2**(epoch+1)) for i in range(num_layers - 1)]
                    vb_hat = [vb[i]/(1-BETA2**(epoch+1)) for i in range(num_layers - 1)]
                    
                    self.weights = {str(i+1):self.weights[str(i+1)] - (learning_rate/(np.sqrt(vw_hat[i])+EPS))*(BETA1*m_w_hat[i]+ (1-BETA1)*dw[i]) for i in range(len(self.weights))} 
                    self.biases = {str(i+1):self.biases[str(i+1)] - (learning_rate/(np.sqrt(vb_hat[i])+EPS))*(BETA1*m_b_hat[i] + (1-BETA1)*db[i]) for i in range(len(self.biases))}

                    dW= [np.zeros((self.layers[l+1], self.layers[l])) for l in range(0, len(self.layers)-1)]
                    db = [np.zeros((self.layers[l+1], 1)) for l in range(0, len(self.layers)-1)]
             
            elapsed = time.time() - start_time

            #y_predict = np.array(y_predict).transpose()
            y_predict = self.predict(self.X_train, self.train_n)
            training_loss.append(np.mean(CE))
            trainingaccuracy.append(self.accuracy(y_train, y_predict, len_dataset)[0])
            validationaccuracy.append(self.accuracy(self.y_val, self.predict(self.X_val, self.val_n), self.val_n)[0])

            print(
                        "Epoch: %d, Loss: %.3e, Training accuracy:%.2f, Validation Accuracy: %.2f, Time: %.2f, Learning Rate: %.3e"
                        % (
                            epoch,
                            training_loss[epoch],
                            trainingaccuracy[epoch],
                            validationaccuracy[epoch],
                            elapsed,
                            self.learning_rate,
                        )
                    )
            wandb.log({'loss':np.mean(CE), 'trainingaccuracy':trainingaccuracy[epoch], 'validationaccuracy':validationaccuracy[epoch],'epoch':epoch })
            
        return training_loss, trainingaccuracy, validationaccuracy, y_predict  


In [ ]:

#Sweep

(trainIn, trainOut), (test_ip, test_op) = fashion_mnist.load_data()

train_n_full = trainOut.shape[0]
train_n = int(0.9*train_n_full)
n_validation = int(0.1 * trainOut.shape[0])
N_test = test_op.shape[0]


idx  = np.random.choice(trainOut.shape[0], train_n_full, replace=False)
idx2 = np.random.choice(test_op.shape[0], N_test, replace=False)

trainInFull = trainIn[idx, :]
trainOutFull = trainOut[idx]

trainIn = trainInFull[:train_n,:]
trainOut = trainOutFull[:train_n]

validIn = trainInFull[train_n:, :]
validOut = trainOutFull[train_n:]    

test_ip = test_ip[idx2, :]
test_op = test_op[idx2]


sweep_config = {
  "name": "Bayesian Sweep",
  "method": "bayes",
  "metric":{
  "name": "validationaccuracy",
  "goal": "maximize"
  },
  "parameters": {
        "maxNo_epoch": {
            "values": [5, 10]
        },

        "initializer": {
            "values": ["RANDOM", "XAVIER", "HE"]
        },

        "num_layers": {
            "values": [2, 3, 4]
        },
        
        
        "hidden_neurons": {
            "values": [32, 64, 128]
        },
        
        "activation": {
            "values": [ 'TANH',  'SIGMOID', 'RELU']
        },
        
        "learning_rate": {
            "values": [0.001, 0.0001]
        },
        
        
        "weight_decay": {
            "values": [0, 0.0005,0.5]
        },
        
        "optimizer": {
            "values": ["SGD", "MGD", "Ng", "RMSPROP", "ADAM","NADAM"]
        },
                    
        "batch_size": {
            "values": [16, 32, 64]
        }
        
        
    }
}

sweep_id = wandb.sweep(sweep_config,project='assi_1_q_4', entity='gadhapremdas2017')



In [ ]:
def train():    
    config_defaults = dict(
            max_epochs=10,
            num_hidden_layers=3,
            num_hidden_neurons=128,
            weight_decay=0,
            learning_rate=1e-3,
            optimizer="NADAM",
            batch_size=32,
            activation="SIGMOID",
            initializer="XAVIER",
            loss="MSE",
        )
        
    #wandb.init(config = config_defaults)
    wandb.init(project='assi_1_mse', entity='gadhapremdas2017',config = config_defaults)


    wandb.run.name = "MSE_hl_" + str(wandb.config.num_hidden_layers) + "_hn_" + str(wandb.config.num_hidden_neurons) + "_opt_" + wandb.config.optimizer + "_act_" + wandb.config.activation + "_lr_" + str(wandb.config.learning_rate) + "_bs_"+str(wandb.config.batch_size) + "_init_" + wandb.config.initializer + "_ep_"+ str(wandb.config.max_epochs)+ "_l2_" + str(wandb.config.weight_decay) 
    CONFIG = wandb.config

    # sweep_id = wandb.sweep(sweep_config)
  

    FFNN = Feed_Forward_NN(
        hidden_layers=CONFIG.hidden_layers,
        hidden_neurons=CONFIG.hidden_neurons,
        Xtrain_ini=trainIn,
        ytrain_ini=trainOut,
        train_n = train_n,
        Xval_ini = validIn,
        yval_ini = validOut,
        val_n = n_validation,
        Xtest_ini = test_ip,
        ytest_ini = test_op,
        N_test = N_test,
        optimizer = CONFIG.optimizer,
        batch_size = CONFIG.batch_size,
        weight_decay = CONFIG.weight_decay,
        learning_rate = CONFIG.learning_rate,
        maxNo_epoch = CONFIG.maxNo_epoch,
        activation = CONFIG.activation,
        initializer = CONFIG.initializer,
        loss = CONFIG.loss
        )



    training_loss, trainingaccuracy, validationaccuracy, y_predict_train = FFNN.optimizer(FFNN.maxNo_epoch, FFNN.train_n, FFNN.batch_size, FFNN.learning_rate)
 

In [ ]:
wandb.agent(sweep_id, train, count = 100)

wandb: Agent Starting Run: 2p0u7imu with config:
wandb: 	activation: TANH
wandb: 	batch_size: 32
wandb: 	initializer: XAVIER
wandb: 	learning_rate: 0.001
wandb: 	max_epochs: 10
wandb: 	num_hidden_neurons: 32
wandb: 	num_layers: 3
wandb: 	optimizer: NAG
wandb: 	weight_decay: 0.0005
wandb: Currently logged in as: gadhapremdas2017. Use `wandb login --relogin` to force relogin


Epoch: 0, Loss: 8.905e-02, Training accuracy:0.80, Validation Accuracy: 0.81, Time: 64.76, Learning Rate: 1.000e-03
Epoch: 1, Loss: 5.210e-02, Training accuracy:0.83, Validation Accuracy: 0.83, Time: 43.08, Learning Rate: 1.000e-03
Epoch: 2, Loss: 4.596e-02, Training accuracy:0.85, Validation Accuracy: 0.85, Time: 42.99, Learning Rate: 1.000e-03
Epoch: 3, Loss: 4.274e-02, Training accuracy:0.85, Validation Accuracy: 0.86, Time: 43.98, Learning Rate: 1.000e-03
Epoch: 4, Loss: 4.055e-02, Training accuracy:0.86, Validation Accuracy: 0.86, Time: 43.00, Learning Rate: 1.000e-03
Epoch: 5, Loss: 3.890e-02, Training accuracy:0.87, Validation Accuracy: 0.87, Time: 51.23, Learning Rate: 1.000e-03
Epoch: 6, Loss: 3.757e-02, Training accuracy:0.87, Validation Accuracy: 0.87, Time: 43.28, Learning Rate: 1.000e-03
Epoch: 7, Loss: 3.645e-02, Training accuracy:0.87, Validation Accuracy: 0.87, Time: 43.15, Learning Rate: 1.000e-03
Epoch: 8, Loss: 3.549e-02, Training accuracy:0.88, Validation Accuracy: 

epoch,▁▂▃▃▄▅▆▆▇█
loss,█▃▂▂▂▂▁▁▁▁
trainingaccuracy,▁▄▅▆▆▇▇▇██
validationaccuracy,▁▄▅▆▇▇▇███
epoch,9
loss,0.03465
trainingaccuracy,0.87991
validationaccuracy,0.87367


wandb: Agent Starting Run: z65g7xx8 with config:
wandb: 	activation: SIGMOID
wandb: 	batch_size: 16
wandb: 	initializer: HE
wandb: 	learning_rate: 0.0001
wandb: 	max_epochs: 10
wandb: 	num_hidden_neurons: 32
wandb: 	num_layers: 2
wandb: 	optimizer: MGD
wandb: 	weight_decay: 0


Epoch: 0, Loss: 2.359e-01, Training accuracy:0.07, Validation Accuracy: 0.07, Time: 44.50, Learning Rate: 1.000e-04
Epoch: 1, Loss: 2.289e-01, Training accuracy:0.29, Validation Accuracy: 0.28, Time: 45.19, Learning Rate: 1.000e-04
Epoch: 2, Loss: 2.278e-01, Training accuracy:0.37, Validation Accuracy: 0.37, Time: 44.14, Learning Rate: 1.000e-04
Epoch: 3, Loss: 2.264e-01, Training accuracy:0.39, Validation Accuracy: 0.39, Time: 44.46, Learning Rate: 1.000e-04
Epoch: 4, Loss: 2.247e-01, Training accuracy:0.39, Validation Accuracy: 0.40, Time: 47.70, Learning Rate: 1.000e-04
Epoch: 5, Loss: 2.224e-01, Training accuracy:0.38, Validation Accuracy: 0.39, Time: 49.16, Learning Rate: 1.000e-04


wandb: Ctrl + C detected. Stopping sweep.


In [ ]:
wandb.finish()

wandb: ERROR Control-C detected -- Run data was not synced
